In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [98]:
X, y = make_blobs(n_samples=240, centers=2, n_features=24, random_state=42)
#plt.plot(X[:, 0][y == 0], X[:, 1][y == 0], 'g^')
#plt.plot(X[:, 0][y == 1], X[:, 1][y == 1], 'bs')
#plt.show()
# taking transpose X for easier calculation
X = X.T
y[y == 0] = -1

In [99]:
class F:
    @staticmethod
    def sign(x):
        if x >= 0:
            return 1
        else:
            return -1

In [100]:
class Perceptron:
    def __init__(self, input_size = 1):
        self.W = np.random.rand(1, input_size)[0]
        self.b = np.random.uniform(0, 3)
    def forward(self, input):
        bias = np.full((1 ,input.shape[1]), self.b)
        return np.matmul(self.W, input) + bias


In [101]:
class Optimizer:
    def __init__(self, model):
        self.model = model
    def update(self, x, y, y_hat):
        if y_hat != y:
            self.model.b += y - y_hat
            self.model.W += (y - y_hat) * x
            return

In [102]:
p = Perceptron(X.shape[0])
optimizer = Optimizer(p)

In [109]:
n_iter = 1000
res = optimizer.model.forward(X)
res = res[0]
print(optimizer.model.W)
print(optimizer.model.b)
for _ in range(n_iter):
    for i in range(X.shape[1]):
        y_hat = F.sign(res[i])
        optimizer.update(X[:, i], y[i], y_hat)
print(optimizer.model.W)
print(optimizer.model.b)

[   24839.56792063   440640.5113482  -1871824.00448329  -206829.68592902
  1065384.65160228 -1493669.20188348  1417573.16268536 -2274263.88460637
 -2284069.06192288  1774679.77045562  3144550.10500644   522897.08568896
 -1606889.24265779 -1346837.26279964  1490969.79201712   302127.18019774
 -1451491.66244044   -52355.72483162 -2070467.93887269  2289706.61807988
 -1388626.25819259  1439628.79679611  -457146.646098     360263.50855644]
142000.69968193927
[   24839.56792063   440640.5113482  -1871824.00448329  -206829.68592902
  1065384.65160228 -1493669.20188348  1417573.16268536 -2274263.88460637
 -2284069.06192288  1774679.77045562  3144550.10500644   522897.08568896
 -1606889.24265779 -1346837.26279964  1490969.79201712   302127.18019774
 -1451491.66244044   -52355.72483162 -2070467.93887269  2289706.61807988
 -1388626.25819259  1439628.79679611  -457146.646098     360263.50855644]
142000.69968193927


In [110]:
miss = 0
for i in range(X.shape[1]):
    y_hat = F.sign(res[i])
    if y_hat != y[i]:
        miss += 1
print(miss)

0


In [111]:
#taking back X to its normal form for easier plotting
#shape of X is in (n, m) form; before transpose, n shows number of features and m shows number of records or datapoints.
X = X.T
X.shape

(240, 24)

In [97]:
def line_calculation(x):
    return -1 * (p.W[0] / p.W[1]) * x - (p.b / p.W[1])

x1_domain = np.linspace(X.min(), X.max(), 100)
x2_domain = line_calculation(x1_domain)
plt.plot(X[:, 0][y == -1], X[:, 1][y == -1], 'g^')
plt.plot(X[:, 0][y == 1], X[:, 1][y == 1], 'bs')
plt.plot(x1_domain, x2_domain, color='red', label='Hyperplane')

plt.xlabel('X-axis')
plt.ylabel('Y-axis')

plt.legend()

plt.show()


IndexError: boolean index did not match indexed array along dimension 0; dimension is 24 but corresponding boolean dimension is 240